# 논문 - 제목, 초록, 키워드 가져오기

## 논문 호출용 클래스 및 함수 정의

In [1]:
import base64
from Crypto.Cipher import AES

class AESTestClass:
    def __init__(self, plain_txt, key):
        # iv, block_size 값은 고정
        self.iv = 'jvHJ1EFA0IXBrxxz'
        self.block_size = 16
        self.plain_txt = plain_txt
        self.key = key

    def pad(self):
        # PKCS#7 패딩
        number_of_bytes_to_pad = self.block_size - len(self.plain_txt) % self.block_size
        ascii_str = chr(number_of_bytes_to_pad)
        padding_str = number_of_bytes_to_pad * ascii_str
        return self.plain_txt + padding_str

    def encrypt(self):
        cipher = AES.new(self.key.encode('utf-8'), AES.MODE_CBC, self.iv.encode('utf-8'))
        padded_txt = self.pad()
        encrypted_bytes = cipher.encrypt(padded_txt.encode('utf-8'))
        # Base64 encrypt
        encrypted_str = base64.urlsafe_b64encode(encrypted_bytes).decode('utf-8')
        return encrypted_str

In [2]:
import xml.etree.ElementTree as ET
import pandas as pd

# XML 형태의 결과를 df로 변경
def xml_to_df(xml):
    root = ET.fromstring(xml)

    # recordList 찾기
    record_list_element = root.find('recordList')

    # 데이터를 담을 리스트
    records = []

    if record_list_element is not None:
        # 각 record에 대해 반복
        for record_element in record_list_element.findall('record'):
            record_data = {}
            # 각 item에 대해 반복
            for item_element in record_element.findall('item'):
                meta_code = item_element.get('metaCode')
                # CDATA 섹션의 텍스트 추출
                value = item_element.text.strip() if item_element.text else ''
                record_data[meta_code] = value
            records.append(record_data)

    df = pd.DataFrame(records)
    return df

In [3]:
import json

# ScienceON에서 사용하는 형태로 쿼리 수정하는 함수
def transform_query(input_query):

    query = {
        "BI": input_query,  # 전체
        # "TI": None,  # 논문명
        # "AU": None,  # 저자
        # "AB": None,  # 초록
        # "KW": None,  # 키워드
        # "PB": None,  # 출판사(발행기관)
        # "SN": None,  # ISSN
        # "BN": None,  # ISBN
        # "PY": None,  # 발행년도
        # "CN": None,  # 문헌번호
        # "DI": None   # DOI
    }

    json_query = json.dumps(query, separators=(',', ':')) 

    return json_query

In [4]:
import json, requests
from datetime import datetime

# 액세스 토큰 불러오기
def call_access_token(MAC_ADDRESS, API_KEY, CLIENT_ID):
    # 맥주소
    mac = (MAC_ADDRESS or "").strip().strip('"').strip("'").upper().replace(":", "-")
    if not mac:
        raise SystemExit("MAC_ADDRESS가 비어있음")

    # datetime 생성
    dt = datetime.now().strftime('%Y%m%d%H%M%S')

    # JSON 페이로드 생성
    payload = {
        "mac_address": mac,
        "datetime": dt
    }
    plain_json = json.dumps(payload, separators=(',', ':'))

    # AES 암호화 → Base64
    aes = AESTestClass(plain_txt=plain_json, key=API_KEY)
    b64_cipher = aes.encrypt()

    # 인코딩 + 토큰 요청
    endpoint = "https://apigateway.kisti.re.kr/tokenrequest.do"
    params = {
        "accounts": b64_cipher,
        "client_id": CLIENT_ID
    }

    response = requests.get(endpoint, params=params, timeout=10)
    response.raise_for_status()

    data = response.json()
    token = data['access_token'] 
    return token

## 예시 데이터

In [5]:
# 예시 데이터
input_id = 'JAKO200411922932805'

## 작동 방식

In [6]:
import os, requests, xmltodict
from dotenv import load_dotenv

load_dotenv(override=True)

CLIENT_ID = os.getenv("SCIENCEON_CLIENT_ID")
ARTI_KEY = os.getenv("SCIENCEON_API_KEY")
MAC_ADDRESS = os.getenv("MAC_ADDRESS")

# access token 생성
access_token = call_access_token(MAC_ADDRESS, ARTI_KEY, CLIENT_ID)

url = "https://apigateway.kisti.re.kr/openapicall.do"
params = {
    "client_id": CLIENT_ID,
    "token": access_token,
    "version": 1.0,
    "action": "browse",
    "target": "ARTI",
    "cn": input_id,
    "include": "",
    "exclude": None,
}

res = requests.get(url, params=params, timeout=20)
xml = res.text
dict_data = xmltodict.parse(xml)

# 데이터를 json으로 저장
with open("../data/input_data.json", "w", encoding="utf-8") as f:
    json.dump(dict_data, f, ensure_ascii=False, indent=4)

df = xml_to_df(xml)

title, description, keyword = df['Title'].iloc[0], df['Abstract'].iloc[0], df['Keyword'].iloc[0]

print('\n[title]\n', title)
print('\n[description]\n', description)
print('\n[keyword]\n', keyword)    



[title]
 한국인의 3차원 무릎관절 구축 및 형상 측정

[description]
 It is necessary to have a model that describes the feature of the knee Joint with a sufficient accuracy. Koreans, however, do not have their own knee joint model to be used in the total knee replacement arthroplasty. They have to use European or American models which do not match Koreans. Three-dimensional visualization techniques are found to be useful in a wide range of medical applications. Three-dimensional imaging studies such as CT(computed tomography) and MRI(magnetic resonance image) provide the primary source of patient-specific data. Three-dimensional knee joint models were constructed by image processing of the CT data of 10 subjects. Using the constructed model, the dimensions of Korean knee joint were measured. And this study proposed a three-dimensional model and data, which can be helpful to develop Korean knee implants and to analyze knee joint movements.

[keyword]
 단층촬영사진 . 한국인 무릎 관절 모델 . 인공 관절 치환술
